# LUZ Rollup

In [2]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

In [3]:
# LUZ Crosswalk 
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=sql2014b8.sandag.org;'
                    'Database=GeoDepot;'
                    'Trusted_Connection=yes;')

with open(r'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\sql_queries\luz_rollup.sql', 'r') as sql_file:
    sql_query = sql_file.read()

df =  pd.read_sql_query(sql_query, conn)
df

,MGRA,LUZ
0,1,95
1,2,95
2,3,95
3,4,95
4,5,95
...,...,...
22997,22998,14
22998,22999,14
22999,23000,14
23000,23001,14


In [20]:
df.describe()

,MGRA,LUZ
count,23002.000000,23002.000000
mean,11501.500000,105.691809
std,6640.249782,63.718692
min,1.000000,1.000000
25%,5751.250000,50.000000
50%,11501.500000,105.000000
75%,17251.750000,160.000000
max,23002.000000,229.000000


In [21]:
categories = ['age', 'ethnicity', 'household_income', 'households', 'housing', 'population', 'sex']


In [4]:
def create_LUZ_output_for_2021_01(category, to_csv):
    data = pd.read_csv(rf'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\files\QA_2021_01_mgra_{category}.csv')

    output = data.merge(df, how='left', left_on='mgra', right_on='MGRA')
    output = output.drop(['mgra', 'MGRA'], axis=1)
    output = output.sort_values(['LUZ', 'yr_id'])
    output = output.groupby(['LUZ', 'yr_id']).sum()
    
    if to_csv == True:
        output.reset_index().to_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\{category}\QA_2021_01_LUZ_{category}.csv', index=False)

    return output
    

In [5]:
def create_LUZ_output_for_age_sex_eth_2021_01(category, to_csv):
    data = pd.read_csv(rf'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\files\QA_2021_01_mgra_{category}.csv')

    output = data.merge(df, how='left', left_on='mgra', right_on='MGRA')
    output = output.drop(['mgra', 'MGRA'], axis=1)
    output = output.sort_values(['LUZ', 'yr_id'])
    output = output.groupby(['LUZ', 'yr_id', 'age_group', 'sex']).sum()
    
    if to_csv == True:
        output.reset_index().to_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\{category}\QA_2021_01_LUZ_{category}.csv', index=False)

    return output

In [28]:
for category in categories:
    create_LUZ_output_for_2021_01(category=category, to_csv=True)
    print(f"{category} is done.")

age is done.
ethnicity is done.
household_income is done.
households is done.
housing is done.
population is done.
sex is done.


In [7]:
create_LUZ_output_for_2021_01(category="households", to_csv=True)

Family household:female householder, no husband present   \
LUZ yr_id                                                             
1   2020                                                1261          
    2021                                                1251          
2   2020                                                1977          
    2021                                                1992          
3   2020                                                3670          
...                                                      ...          
227 2021                                                  28          
228 2020                                                   3          
    2021                                                   3          
229 2020                                                 335          
    2021                                                 335          

           Family household:male householder, no wife present  \
LUZ yr_id                                                       
1   2020                                                2766    
    2021                                                2843    
2   2020                                                3333    
    2021                                                3450    
3   2020                                                6067    
...                                                      ...    
227 2021                                                  41    
228 2020                                                   5    
    2021                                                   4    
229 2020                                                 331    
    2021                                                 358    

           Family household:married-couple  \
LUZ yr_id                                    
1   2020                              2337   
    2021                              2316   
2   2020                              2088   
    2021                              2077   
3   2020                              3744   
...                                    ...   
227 2021                                14   
228 2020                                 4   
    2021                                 4   
229 2020                               114   
    2021                               121   

           Nonfamily household:female householder, living alone  \
LUZ yr_id                                                         
1   2020                                                 295      
    2021                                                 272      
2   2020                                                 454      
    2021                                                 440      
3   2020                                                 928      
...                                                      ...      
227 2021                                                  11      
228 2020                                                   5      
    2021                                                   2      
229 2020                                                 164      
    2021                                                 150      

           Nonfamily household:female householder, not living alone  \
LUZ yr_id                                                             
1   2020                                                 235          
    2021                                                 234          
2   2020                                                 282          
    2021                                                 291          
3   2020                                                 521          
...                                                      ...          
227 2021                                                  12          
228 2020                                                   3          
    2021                                                   5

In [6]:
create_LUZ_output_for_age_sex_eth_2021_01(category='age_sex_ethnicity', to_csv=True)

Hispanic  \
LUZ yr_id age_group    sex                
1   2020  10 to 14     Female       343   
                       Male         314   
          15 to 17     Female       210   
                       Male         249   
          18 and 19    Female       101   
...                                 ...   
229 2021  80 to 84     Male          19   
          85 and Older Female        35   
                       Male          20   
          Under 5      Female       158   
                       Male         195   

                               Non-Hispanic, American Indian or Alaska Native  \
LUZ yr_id age_group    sex                                                      
1   2020  10 to 14     Female                                               5   
                       Male                                                 3   
          15 to 17     Female                                               1   
                       Male                                                 2   
          18 and 19    Female                                               1   
...                                                                       ...   
229 2021  80 to 84     Male                                                 0   
          85 and Older Female                                               0   
                       Male                                                 0   
          Under 5      Female                                               1   
                       Male                                                 1   

                               Non-Hispanic, Asian  Non-Hispanic, Black  \
LUZ yr_id age_group    sex                                                
1   2020  10 to 14     Female                   30                   16   
                       Male                     24                   23   
          15 to 17     Female                    8                    8   
                       Male                     18                   19   
          18 and 19    Female                    5                    8   
...                                            ...                  ...   
229 2021  80 to 84     Male                      2                    0   
          85 and Older Female                    3                    1   
                       Male                      2                    4   
          Under 5      Female                    7                    6   
                       Male                      6                    9   

                               Non-Hispanic, Hawaiian or Pacific Islander  \
LUZ yr_id age_group    sex                                                  
1   2020  10 to 14     Female                                           0   
                       Male                                             5   
          15 to 17     Female                                           3   
                       Male                                             2   
          18 and 19    Female                                           1   
...                                                                   ...   
229 2021  80 to 84     Male                                             1   
          85 and Older Female                                           1   
                       Male                                             0   
          Under 5      Female                                           0   
                       Male                                             0   

                               Non-Hispanic, Other  \
LUZ yr_id age_group    sex                           
1   2020  10 to 14     Female                    2   
                       Male                      2   
          15 to 17     Female                    2   
                       Male                      1   
          18 and 19    Female                    2   
...                                            ...   
229 202

In [15]:
data = pd.read_csv(r'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\files\QA_2021_01_mgra_age.csv')
data

,mgra,yr_id,Under 5,5 to 9,10 to 14,15 to 17,18 and 19,20 to 24,25 to 29,30 to 34,...,45 to 49,50 to 54,55 to 59,60 and 61,62 to 64,65 to 69,70 to 74,75 to 79,80 to 84,85 and Older
0,1,2010,2,3,3,3,0,0,1,2,...,4,7,3,1,2,0,1,2,3,0
1,1,2011,1,3,4,3,0,0,0,2,...,4,8,3,0,3,0,1,1,3,1
2,1,2012,1,4,4,1,2,0,0,1,...,5,7,4,1,2,1,1,1,3,1
3,1,2013,1,5,3,1,4,1,0,1,...,4,6,4,2,1,2,1,1,2,1
4,1,2014,1,8,3,1,1,3,3,1,...,4,4,5,2,0,3,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276019,23002,2017,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276020,23002,2018,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276021,23002,2019,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276022,23002,2020,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
output = data.merge(df, how='left', left_on='mgra', right_on='MGRA')
output = output.drop(['mgra', 'MGRA'], axis=1)
output = output.sort_values(['LUZ', 'yr_id'])
output = output.groupby(['LUZ', 'yr_id']).sum()
output

Under 5  5 to 9  10 to 14  15 to 17  18 and 19  20 to 24  25 to 29  \
LUZ yr_id                                                                       
1   2010      1233    1011       874       676        473      1487      1799   
    2011      1331    1084       969       654        474      1455      1756   
    2012      1376    1166      1079       684        481      1538      1711   
    2013      1455    1299      1148       756        481      1614      1667   
    2014      1454    1386      1143       764        452      1642      1579   
...            ...     ...       ...       ...        ...       ...       ...   
229 2017       539     566       530       348        202       595       439   
    2018       518     583       568       355        212       556       445   
    2019       497     580       576       323        205       582       440   
    2020       471     567       572       316        181       553       446   
    2021       460     596       551       341        185       513       424   

           30 to 34  35 to 39  40 to 44  45 to 49  50 to 54  55 to 59  \
LUZ yr_id                                                               
1   2010       1556      1338      1206      1321      1351      1190   
    2011       1627      1289      1298      1297      1373      1243   
    2012       1687      1270      1334      1326      1395      1351   
    2013       1825      1374      1344      1322      1423      1370   
    2014       1786      1477      1335      1318      1394      1418   
...             ...       ...       ...       ...       ...       ...   
229 2017        410       382       368       378       356       336   
    2018        384       392       351       370       365       337   
    2019        405       391       341       354       347       335   
    2020        372       397       335       334       351       322   
    2021        362       366       334       326       348       298   

           60 and 61  62 to 64  65 to 69  70 to 74  75 to 79  80 to 84  \
LUZ yr_id                                                                
1   2010         402       493       599       463       375       298   
    2011         406       576       642       470       379       322   
    2012         429       629       721       505       417       332   
    2013         500       617       820       552       440       343   
    2014         533       639       891       590       451       346   
...              ...       ...       ...       ...       ...       ...   
229 2017         113       138       190       145        86        70   
    2018         128       138       201       149        91        74   
    2019         119       155       196       168        98        71   
    2020         108       159       189       150       101        72   
    2021         108       152       201       178       103        68   

           85 and Older  
LUZ yr_id                
1   2010            255  
    2011            291  
    2012            321  
    2013            351  
    2014            372  
...                 ...  
229 2017             70  
    2018             70  
    2019             67  
    2020             68  
    2021             68  

[2748 rows x 20 columns]